# Prediction with LSTM model v01
<p style='text-align: right;'>with selectd3.csv</p>

* history
  * 2025/05/16 PM06:56 : 4번째 모델의 8번째 데이터를 사용

In [7]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [8]:
restart_kernel()

restart kernel... 2025-05-15 17:06:07.360561


In [3]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from globalvar import *
from data import DataLoader
from model1 import PredictLSTM1
from model2 import PredictLSTM2
from model3 import PredictLSTM3
from model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from util_pred import print_data
from util_pred import flat_data_with_sum, flat_data, get_frequency
from util_pred import save_model, import_mode
from util_pred import get_random_in_list
from util_pred import print_data_with_sort
from util_pred import get_sorted_n_values
from util_pred import dict_key_count
from util_pred import print_list
from util_pred import print_dict_list
from util_pred import print_title
from activation import ActivationOutput, RecurrentActivation
from datetime import datetime

from operate import create_model, create_model_v1, get_predicted

print(f"imported library. ({datetime.now()})")

imported library. (2025-05-16 18:14:30.575871)


In [27]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./selectd3.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[7, 9, 24, 40, 42, 44],
        [3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 6

finished to set environemnt. (2025-05-19 10:52:48.827199)


In [ ]:
def build_and_compile_lstm_model(
    self,
    seq_len: int,
    n_features: int,
    lstm_units: list,
    learning_rate: float,
    dropout: float,
    steps: int,
    metrics: str,
    single_output: bool,
    last_lstm_return_sequences: bool = False,
    dense_units: list = None,
    activation: str = None,
):
    """
    LSTM 네트워크를 생성한 결과를 반환한다.

    :param seq_len: Length of sequences. (Look back window size)
    :param n_features: Number of features. It requires for model input shape.
    :param lstm_units: Number of cells each LSTM layers.
    :param learning_rate: Learning rate.
    :param dropout: Dropout rate.
    :param steps: Length to predict.
    :param metrics: Model loss function metric.
    :param single_output: Whether 'yhat' is a multiple value or a single value.
    :param last_lstm_return_sequences: Last LSTM's `return_sequences`. Allow when `single_output=False` only.
    :param dense_units: Number of cells each Dense layers. It adds after LSTM layers.
    :param activation: Activation function of Layers.
    """
    tf.random.set_seed(self.random_seed)
    model = Sequential()

    if len(lstm_units) > 1:
        # LSTM -> ... -> LSTM -> Dense(steps)
        model.add(
            LSTM(
                units=lstm_units[0],
                activation=activation,
                return_sequences=True,
                input_shape=(seq_len, n_features),
            )
        )
        lstm_layers = lstm_units[1:]
        for i, n_units in enumerate(lstm_layers, start=1):
            if i == len(lstm_layers):
                if single_output:
                    return_sequences = False
                else:
                    return_sequences = last_lstm_return_sequences
                model.add(
                    LSTM(
                        units=n_units,
                        activation=activation,
                        return_sequences=return_sequences,
                    )
                )
            else:
                model.add(
                    LSTM(
                        units=n_units,
                        activation=activation,
                        return_sequences=True,
                    )
                )
    else:
        # LSTM -> Dense(steps)
        if single_output:
            return_sequences = False
        else:
            return_sequences = last_lstm_return_sequences
        model.add(
            LSTM(
                units=lstm_units[0],
                activation=activation,
                return_sequences=return_sequences,
                input_shape=(seq_len, n_features),
            )
        )

    if single_output:  # Single Step, Direct Multi Step
        if dense_units:
            for n_units in dense_units:
                model.add(Dense(units=n_units, activation=activation))
        if dropout > 0:
            model.add(Dropout(rate=dropout))
        model.add(Dense(1))
    else:  # Multiple Output Step
        if last_lstm_return_sequences:
            model.add(Flatten())
        if dense_units:
            for n_units in dense_units:
                model.add(Dense(units=n_units, activation=activation))
        if dropout > 0:
            model.add(Dropout(rate=dropout))
        model.add(Dense(units=steps))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=MSE, metrics=metrics)
    return model


In [ ]:
self.model = self.build_and_compile_lstm_model(
        seq_len=seq_len,
        n_features=n_features,
        lstm_units=lstm_units,
        activation=activation,
        learning_rate=learning_rate,
        dropout=dropout,
        steps=steps,
        last_lstm_return_sequences=last_lstm_return_sequences,
        dense_units=dense_units,
        metrics=metrics,
        single_output=single_output,
    )


In [5]:
# data prepare and set base model
dataset = DataLoader(data_dir=data_dir,
                     training_length=training_length,
                     window_prev=window,
                     mode=mode,
                     from_pos=from_pos
                    )

dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300
                  )
    }
print(f'completed to load data. {datetime.now()}')

DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]]
rDataLoader.preproc_csv aw_np_proc = 571, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [12 24 26 39 42]
 [13 28 34 38 42]
 [ 6  7 11 12 17]]
DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]
 [1171    3    6 ...   11   12   17]]
rDataLoader.preproc_csv aw_np_proc = 571, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [12 24 26 39 42]
 [13 28 34 38 42]
 [ 6  7 11 12 17]]
DataLoader.preproc_csv origin = 571, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [116

In [6]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
lstm_layers = {
    4: [LSTM(hid_dim,
             activation=ActivationOutput.selu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.sigmoid.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],
    }

simple_lstm_layer = [LSTM(hid_dim,
                     activation=ActivationOutput.elu.name)]

dense_layers = {0: [Dense(45, activation='softmax'),
                    Dense(45, activation='sigmoid')],
                7: [Dense(45, activation='softmax')]
               }
simple_dense_layer = [Dense(45, activation='softmax')]

layers = [
    lstm_layers[4] + dense_layers[7], # 4
    lstm_layers[4] + dense_layers[0], # 5
    lstm_layers[4] + dense_layers[7], # 4
    lstm_layers[4] + dense_layers[0], # 5
    lstm_layers[4] + dense_layers[7], # 4
    lstm_layers[4] + dense_layers[0], # 5
    ]

datasets = [dataset_dicts[1], # 4
            dataset_dicts[2], # 5
            dataset_dicts[1], # 4
            dataset_dicts[2], # 5
            dataset_dicts[1], # 4
            dataset_dicts[2], # 5
           ]

lstm_args = [
    {
            "lstm_units=lstm_units,
            "learning_rate=learning_rate,
            "dropout=dropout,
            "steps=steps,
            "metrics=metrics,
            "last_lstm_return_sequences=False,
            "dense_units=[45],
            "activations=["softmax"],
            "dense_activation=["softmax"],
            "output_dense_activation=output_dense_activation
    }
]

epochs = [20, 10, 25, 25, 100, 100]

models = [None for i in range(MAX_MODEL_CNT)]
#matched_cnts = [0 for i in range(MAX_MODEL_CNT)]
#selected_fives = [None for i in range(MAX_MODEL_CNT)]
#matched_list = [None for i in range(MAX_MODEL_CNT)]
#predicted_all = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-16 18:14:42.434422


In [7]:
# all : create model and training
print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model(id=i,
                         model_type="lstm4",
                         layers=layers[i],
                         dataset=datasets[i],
                         hid_dim=hid_dim,
                         epoch=epochs[i],
                         verbose=False
                        )
    models[i] = (model, layers[i])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')

start to train all models. 2025-05-16 18:14:47.539282
1's training. status=start 2025-05-16 18:14:47.539459
1's training. status=end 2025-05-16 18:15:15.473533
2's training. status=start 2025-05-16 18:15:15.473669
2's training. status=end 2025-05-16 18:15:24.904533
3's training. status=start 2025-05-16 18:15:24.904646
3's training. status=end 2025-05-16 18:16:00.316994
4's training. status=start 2025-05-16 18:16:00.317133
4's training. status=end 2025-05-16 18:16:17.799789
5's training. status=start 2025-05-16 18:16:17.799893
5's training. status=end 2025-05-16 18:18:30.151002
6's training. status=start 2025-05-16 18:18:30.151112
6's training. status=end 2025-05-16 18:19:34.904407
completed to train all models. 2025-05-16 18:19:34.905147


In [107]:
print(f'check to create model. {datetime.now()}')

check to create model. 2025-05-16 16:59:31.699518


In [ ]:
# all : predict #1
greed_prediction = [None for i in range(MAX_MODEL_CNT)]
random_pred = [None for i in range(MAX_MODEL_CNT)]
for i in range(MAX_MODEL_CNT):
    greed_prediction[i] = models[i][0].predict_numbers("greed", trial=1)
    random_pred[i] = models[i][0].predict_randomely(trial=1)
    print(f'{i+1}\'s greed_prediction = {greed_prediction[i]}')
    print(f'{i+1}\'s random_pred = {random_pred[i]}')
print(f'comleted to predict model. {datetime.now()}')

In [ ]:
matched_cnts, selected_fives, matched_list, predicted_all = repeat_predict_all_model(trial=20)

In [ ]:
print(matched_cnts)
print(matched_list)

In [10]:
import json
import os
from datetime import datetime

def listinlist_2_strinlist(list_in_list):
    """ listinlist_2_strinlist """
    str_in_list = []
    for l in list_in_list:
        str_in_list.append(",".join(l))
    return str_in_list      


def change_matched_info(matched_count, matched_list):
    counts = {}
    cnt = 1
    for c in matched_count:
        counts[cnt] = ",".join([ str(i) for i in c])
        cnt += 1
    cnt = 1
    datas = {}
    for d in matched_list:
        datas[cnt] = d
        cnt += 1
    return counts, datas

def write_json(trial, matched_count, matched_list, append=False):
    suffix = datetime.now().strftime('%y%m%d_%H')
    file_name = f"matched_{suffix}.json"
    matched_dict = {}
    if append:
        if os.path.isfile(file_name):
            with open(file_name, "r") as file:
                matched_dict = json.load(file)
                file.close()
    counts, datas = change_matched_info(matched_count, matched_list)
    if len(counts) > 0 and len(datas) > 0:
        matched_dict[trial] = {}
        matched_dict[trial]["counts"] =  counts
        matched_dict[trial]["datas"] = datas
        print(f'matched_dict = {matched_dict}')
        with open(file_name, "w") as file:
            json.dump(matched_dict, file, indent=4)

In [ ]:
# all with trial : print predicted
write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
print_dict_list(title="matched_cnt", datas=matched_cnts)
for i in range(MAX_MODEL_CNT):
    print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

| 회차 | 3이상 ||| 0이 없음 |
| -- | -- | -- | -- | -- |
|  1 | N ||| 2 |
|  2 | N ||| 1 |
|  3 | **1**, 2 ||| **1** |
|  4 | N ||| N |
|  5 | N ||| N |
|  6 | N ||| N |
|  7 | N ||| N |
|  8 | 2 ||| N |

|  8 | 1 ||| N |
|  8 | N ||| N |
|  8 | N ||| N |
|  8 | 2 ||| N |




{14:6, 13:8, 12:4, _11:10_, 10:3, 9:1, 8:6, 7:5, 6:4, 5:9, 4:4, 3:7, 2:7, 1:4}
## 4시 40분경 데이터
- {2: 6, 5: 5, 6: 4, 4: 3, 3: 3, 1: 2}
- {6: 4, 5: 5, 4: 3, 3: 3, 2: 6, 1: 2}

## 5시 20분경 데이터
- {4: 5, 5: 5, 1: 4, 6: 3, 3: 2}
- {6: 3, 5: 5, 4: 5, 3: 2, 1: 4}

In [28]:
# all (SMALL): predict #2
def predict_small(trial, last, models):
    matched_cnts_small = [0 for i in range(MAX_MODEL_CNT)]
    selected_fives_small = [None for i in range(MAX_MODEL_CNT)]
    matched_list_small = [None for i in range(MAX_MODEL_CNT)]
    predicted_all_small = [None for i in range(MAX_MODEL_CNT)]

    for i in range(MAX_MODEL_CNT):
        title = f"history #{i+1}"
        model_num = i
        if models[model_num] is None:
            print(f"model #{model_num+1} is None")
        else:
            models[model_num][0].verb = "None"
            matched_cnts_small[model_num], \
            selected_fives_small[model_num], \
            matched_list_small[model_num], \
            predicted_all_small[model_num] = get_predicted(
                title=title,
                model=models[model_num][0],
                mode=mode2,
                use_pre=False,
                last=last,
                verbose=False,
                trial=5
            )
    print(f'predicted time : trial={trial} date={datetime.now()}')
    return matched_cnts_small, matched_list_small

In [29]:
# all (SMALL): print predicted
def print_predict_small(matched_cnts_small, matched_list_small):
    print(f'print time : {datetime.now()}')
    #write_json(trial = 3, matched_count=matched_cnts_small, matched_list=matched_list_small, append=True)
    print_dict_list(title="matched_cnt", datas=matched_cnts_small)
    for i in range(MAX_MODEL_CNT):
        print_list(title=f"matched_list[{i+1}]", datas=matched_list_small[i])

In [32]:
trial = 1
matched_cnts_all = []
matched_list = []
for t in range(1, trial+1):
    matched_cnts_small, matched_list_small = predict_small(trial=t, last=last, models=models)
    #print_predict_small(matched_cnts_small, matched_list_small)
    matched_cnts_all.append(matched_cnts_small)
    matched_list.append(matched_list_small)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
predicted time : trial=1 date=2025-05-19 10:54:28.092458


In [36]:
cnt = 1
for matched in matched_list:
    cnt1 = 1
    for m in matched:
        print_list(title=f"total_matched_{cnt}_model_{cnt1}", datas=m)
        cnt1 += 1
    cnt += 1

total_matched_1_model_1
	([16, 25, 36, 40, 41, 44], [40, 44])
	([29, 31, 5, 44, 18, 38], [44])
	([26, 23, 30, 36, 21, 2], [])
	([7, 15, 21, 22, 37, 42], [7, 42])
	([3, 11, 18, 30, 35, 8], [])
total_matched_1_model_2
	([6, 16, 23, 32, 44, 45], [44])
	([17, 41, 18, 35, 31, 12], [])
	([6, 12, 15, 17, 32, 38], [])
	([28, 4, 31, 39, 2, 21], [])
	([5, 19, 23, 39, 42, 44], [42, 44])
total_matched_1_model_3
	([30, 1, 18, 5, 7, 41], [7])
	([2, 3, 6, 14, 24, 45], [24])
	([2, 7, 13, 14, 27, 36], [7])
	([19, 14, 22, 6, 23, 45], [])
	([8, 19, 34, 35, 40, 42], [40, 42])
total_matched_1_model_4
	([1, 11, 12, 15, 21, 36], [])
	([11, 31, 34, 36, 37, 42], [42])
	([2, 6, 7, 16, 40, 45], [7, 40])
	([29, 36, 33, 44, 2, 7], [7, 44])
	([5, 7, 22, 30, 37, 41], [7])
total_matched_1_model_5
	([2, 15, 22, 33, 37, 40], [40])
	([6, 4, 33, 42, 17, 29], [42])
	([2, 16, 23, 30, 38, 45], [])
	([26, 41, 44, 16, 2, 36], [44])
	([38, 41, 11, 5, 12, 28], [])
total_matched_1_model_6
	([29, 2, 30, 23, 42, 6], [42])
	([8, 16

In [ ]:
([3, 5, 10, 11, 13, 26], [3, 11]),
([36, 10, 7, 32, 1, 11], [7, 11]),
([20, 41, 21, 14, 6, 40], [6]),
([12, 15, 19, 21, 34, 36], [12]),
([27, 40, 35, 5, 45, 37], [])]


([3, 9, 11, 28, 29, 30], [3, 11]),
([1, 6, 12, 22, 28, 32], [6, 12]),
([1, 11, 25, 28, 32, 45], [11]),
([7, 22, 27, 28, 38, 39], [7]),
([39, 12, 24, 11, 31, 25], [11, 12])


([3, 4, 8, 21, 26, 27], [3]),
([3, 4, 14, 18, 28, 37], [3]),
([3, 15, 17, 23, 33, 36], [3, 17]),
([44, 30, 5, 41, 10, 3], [3]),
([1, 6, 13, 17, 30, 45], [6, 17])


In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [55]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [ ]:
# predict #1
title = "history #1"
model_num = 0 # real 1
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #2
title = "history #2"
model_num = 1 # real 2
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #3
title = "history #3"
model_num = 2 # real 3
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #4
title = "history #4"
model_num = 3 # real 4
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #5
title = "history #5"
model_num = 4 # real 5
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #6
title = "history #6"
model_num = 5 # real 6
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [22]:
# [3, 6, 7, 11, 12, 17]

In [29]:
new1 = flat_data(prediction_number_set22)
new_dict = {}
for n in new1:
    if n in new_dict:
        new_dict[n] += 1
    else:
        new_dict[n] = 1
# print(f'new_dict = {new_dict}')
top, all = get_sorted_n_values(new_dict)
print(f'all = {all}')
print(f'top = {top}')
# print(f'new_dict = {new_dict}')
bottom, all = get_sorted_n_values(new_dict, is_sort=False)
print(f'bottom = {bottom}')

NameError: name 'prediction_number_set22' is not defined

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]